In [16]:
pyautogui.FAILSAFE = True

import pyautogui
import os

# get player names from MCP-Records/data-analysis/in-new folder
directory = "/Users/eliasmb/VScodeProjects/MCP-Records/MCP-Records/data-analysis/in-new"
file_names = []

for file in os.listdir(directory):
    if file.endswith(".txt") and "log" not in file:
        file_names.append(file[:-4])

# List of players, source from directory with previously scraped player names 
playerList = sorted(file_names)

# List of Positions for each record icon x, y (at given GUI scale, resolution (4k), window size) 
_4kRecordIconPositionsX = [1842, 2274, 2274, 1842, 1842, 2274]
_4kRecordIconPositionsY = [1155, 1155, 1209, 1209, 1260, 1260]

_2kRecordIconPositionsX = [921, 1137, 1137, 921, 921, 1137]
_2kRecordIconPositionsY = [int(i) / 2 for i in _4kRecordIconPositionsY]

# wait 5s before starting
pyautogui.sleep(5)

# Scrape Records for Each Player in playerList
for player in playerList:
    # Switch to Minecraft window
    # minecraft_window = pyautogui.getWindowsWithTitle("minecraft")[0] 
    # eminecraft_window.activate()

    # Open Chat
    pyautogui.press("t")

    # Assemble Command String
    command = "/rekorde " + player

    # Execute Command
    pyautogui.typewrite(command)
    pyautogui.press("enter")

    # mooth smoothly through all positions x, y, taking x and y from the respective list

    pyautogui.moveTo(x = 500, y = 500, duration=0.2)

    for i in range(0, 6):
        pyautogui.moveTo(x = _2kRecordIconPositionsX[i], y = _2kRecordIconPositionsY[i], duration=0.3)
        # pyautogui.click()
        pyautogui.sleep(0.1)

        pyautogui.moveTo()

    pyautogui.press("e")

    # Wait 300 ms until next player] 
    pyautogui.sleep(0.3)

    # Rename file on desktop
    new_filename = player + ".txt"
    desktop_path = os.path.expanduser("~/Desktop")
    old_filepath = os.path.join(desktop_path, "ItemTooltipExport.txt")
    new_filepath = os.path.join(desktop_path, new_filename)
    os.rename(old_filepath, new_filepath)

    print("Record for " + player + " scraped successfully." + "\n")

TypeError: 'int' object is not callable